In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_json(r"E:\MYLEARN\2-ANALYTICS-DataScience\datasets\NLP\recipes.json")


In [3]:
data.head()

,Day,Ingredients,Instructions,Month,Name,Url,Weekday,Year
0,1,"[600 g Hackfleisch, halb und halb, 800 g Sauer...",Die Eier hart kochen. Dann pellen und mit eine...,January,Gebratener Hasenrücken,https://www.chefkoch.de/rezepte/185441079701305/,Thursday,2009
1,1,"[1 kg Strauchtomate(n), 1 Gemüsezwiebel(n), 1 ...",Vorab folgende Bemerkung: Alle Mengen sind Cir...,July,Pilz Stroganoff,https://www.chefkoch.de/rezepte/2718181424631245/,Saturday,2017
2,1,"[1 Glas Kirsche(n), 1 Pck. Vanillepuddingpulve...","Die Kirschen abtropfen lassen, dabei den Saft ...",January,Kaninchen a la Gioff,https://www.chefkoch.de/rezepte/46341015661368/,Monday,2007
3,1,"[500 g Spargel, grüner, 300 ml Brühe oder Fond...","Den Spargel säubern, die holzigen Enden abschn...",April,Spanisches Knoblauch-Kaninchen,https://www.chefkoch.de/rezepte/51051018014178/,Monday,2013
4,1,"[250 g Kohlrabi, 150 ml Gemüsebrühe, 150 ml Mi...",Kohlrabi schälen und klein würfeln. Mit der Br...,August,Gnocchi - Hack - Pfanne mit Basilikum,https://www.chefkoch.de/rezepte/1555771262860408/,Tuesday,2017


In [4]:
instructions = data.Instructions

In [5]:
import spacy

In [6]:
nlp = spacy.load('de_core_news_sm', disable=['parser', 'tagger', 'ner'])

In [8]:
tokenized = nlp.tokenizer.pipe(instructions.values)

tokenized = [t.text for doc in tokenized for t in doc]

In [9]:
tokenized[:10]

['Die', 'Eier', 'hart', 'kochen', '.', 'Dann', 'pellen', 'und', 'mit', 'einem']

## Create tri-grams

In [10]:
from collections import Counter, defaultdict
from itertools import tee

In [11]:
def make_trigrams(iterator):
    a, b, c = tee(iterator, 3)
    next(b)
    next(c)
    next(c)
    
    return zip(a, b, c)

In [12]:
trigrams = Counter(make_trigrams(tokenized))

In [13]:
trigrams.most_common(10)

[(('Salz', 'und', 'Pfeffer'), 4400),
 (('Salz', ',', 'Pfeffer'), 2996),
 (('mit', 'Salz', 'und'), 2765),
 (('.', 'Mit', 'Salz'), 2540),
 ((',', 'Pfeffer', 'und'), 2169),
 (('in', 'einer', 'Pfanne'), 1952),
 (('und', 'Pfeffer', 'würzen'), 1831),
 (('lassen', '.', 'Die'), 1742),
 (('schneiden', '.', 'Die'), 1710),
 (('köcheln', 'lassen', '.'), 1669)]

## Build a next-word-lookup

In [14]:
d = defaultdict(Counter)

for a, b, c in trigrams:
        
    d[a, b][c] += trigrams[a, b, c]
    
    #print(a,b,c)


In [15]:
i= 1
for k, v in d.items():
    if i> 10:
        break
    print(k, ':', v)
    i +=1

('Die', 'Eier') : Counter({'mit': 139, 'pochieren': 121, 'trennen': 50, 'in': 41, 'hart': 32, 'verquirlen': 29, ',': 19, 'und': 14, 'nacheinander': 7, 'ca.': 6, 'schälen': 6, 'verschlagen': 5, ':': 4, 'kräftig': 4, 'etwa': 4, 'im': 4, 'darauf': 4, 'einzeln': 4, 'pellen': 4, 'schlagen': 3, 'darübergießen': 3, 'aufschlagen': 3, '-': 3, 'wachsweich': 3, 'dazu': 3, 'nach': 3, 'hartkochen': 2, 'klein': 2, 'dabei': 2, 'dazugeben': 2, 'zu': 2, 'abgießen': 2, 'leicht': 1, 'inzwischen': 1, 'mir': 1, 'werden': 1, 'abkochen': 1})
('Eier', 'hart') : Counter({'kochen': 49})
('hart', 'kochen') : Counter({',': 33, '.': 24, 'und': 10})
('kochen', '.') : Counter({'Die': 188, 'Das': 135, 'In': 106, 'Den': 91, 'Mit': 71, 'Inzwischen': 68, 'Dann': 61, 'Nach': 59, 'Abgießen': 54, 'Fischfilet': 29, 'Anschließend': 29, 'Danach': 27, 'Zum': 27, 'Beim': 27, 'Bohnen': 18, 'Für': 17, 'Pilze': 17, 'Abschrecken': 16, 'Abtropfen': 15, 'Möhren': 14, 'Dabei': 14, 'Währenddessen': 13, 'Speck': 12, 'Kartoffeln': 11, 'W

look up the next word for “die Möhrensuppe”.

In [16]:
d["die", "Möhrensuppe"]

Counter({'geben': 7, 'die': 2, 'ringsum': 2})

In [17]:
d["die", "Möhrensuppe"].elements()

In [18]:
list(d["die", "Möhrensuppe"].elements())

['geben',
 'geben',
 'geben',
 'geben',
 'geben',
 'geben',
 'geben',
 'die',
 'die',
 'ringsum',
 'ringsum']